In [ ]:
import matplotlib.pyplot as plt
from matplotlib import figure
import gudhi
import laspy as lp
import numpy as np
import os
import glob 

os.chdir("/Users/oliviachandrasekhar/Desktop/Research/s21_internship/data/tls_data/t1/thinned")
list = glob.glob('*.las')
print(list)
len(list)

num_tiles=len(list)
tiles= []
#for loop to read in data, convert to np arrays
for i in range(num_tiles):
    file=lp.file.File(list[i], mode="r")
    file_array=np.array((file.x, file.y, file.z)).transpose()
    tiles.append(file_array)

#for one tile at a time--could also do a for loop
tile=tiles[0] #specify tile
max_alpha=.05 #can set max_alpha_square
ac = gudhi.AlphaComplex(tile)
st = ac.create_simplex_tree(max_alpha) 

#get vertices of the triangles as vertices of the simplicial complex
points = np.array([ac.get_point(i) for i in range(st.num_vertices())])
# We are only going to plot the triangles
triangles = np.array([s[0] for s in st.get_skeleton(2) if len(s[0]) == 3 and s[1] <= 0.5])

# Visualization with plotly---this code is not due to me

import plotly
from plotly.graph_objs import graph_objs as go
import ipywidgets as widgets

plotly.offline.init_notebook_mode()
from plotly.offline import iplot

alpha = widgets.FloatSlider(
    value = 0.005,
    min = 0.0,
    max = 0.01,
    step = 0.0001,
    description = 'Alpha:', 
    readout_format = '.4f'
)

mesh = go.Mesh3d(
    x = points[:, 0], 
    y = points[:, 1], 
    z = points[:, 2], 
    i = triangles[:, 0], 
    j = triangles[:, 1], 
    k = triangles[:, 2]
)

fig = go.FigureWidget(
    data = mesh, 
    layout = go.Layout(
        title = dict(
            text = 'Alpha Complex Representation'
        ), 
        scene = dict(
            xaxis = dict(nticks = 4, range = [-4, 4]), 
            yaxis = dict(nticks = 4, range = [-4, 4]), 
            zaxis = dict(nticks = 4, range = [-4, 4])
        )
    )
)
def view_torus(alpha):
    if alpha < 0.0015:
        alpha = 0.0015
    triangles = np.array([s[0] for s in st.get_skeleton(2) if len(s[0]) == 3 and s[1] <= alpha])
    fig.data[0].i = triangles[:, 0]
    fig.data[0].j = triangles[:, 1]
    fig.data[0].k = triangles[:, 2]
    iplot(fig)

widgets.interact(view_torus, alpha = alpha);
